# Tasca M6 T01
Exercicis per practicar amb regressions.

## Exercici 1
Crea almenys dos models de regressió diferents per intentar predir el millor possible el preu de les vivendes (MEDV) de l'arxiu adjunt.

In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import statsmodels.api as sm
from sklearn import linear_model

from sklearn.metrics import mean_squared_error
import math

In [2]:
filePath = r"C:\Formación\2022 BarcelonaActiva\CursDSOnline\Sprint06_aprenentatge_supervisat" + os.sep
file = filePath + "housing data.csv"
dfBoston = pd.read_csv(file)
dfBoston.head(4)

,0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98,24.00
0,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
1,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
2,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
3,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [3]:
# Reanomeno les columnes del dataframe perquè siguin més descriptives
dfBoston.columns= ['CRIM', 'ZN', 'INDUS','CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
dfBoston.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')

In [4]:
dfBoston.columns = [col.lower() for col in dfBoston.columns]

### Anàlisi descriptiu de les dades

In [5]:
dfBoston.head(4)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
1,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
2,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
3,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [6]:
# Tipus de dades de cada columna
dfBoston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   crim     505 non-null    float64
 1   zn       505 non-null    float64
 2   indus    505 non-null    float64
 3   chas     505 non-null    int64  
 4   nox      505 non-null    float64
 5   rm       505 non-null    float64
 6   age      505 non-null    float64
 7   dis      505 non-null    float64
 8   rad      505 non-null    int64  
 9   tax      505 non-null    float64
 10  ptratio  505 non-null    float64
 11  b        505 non-null    float64
 12  lstat    505 non-null    float64
 13  medv     505 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.4 KB


In [7]:
# Dimensions del dataframe
dfBoston.shape

(505, 14)

In [8]:
# Nombre de valors absents per variable
dfBoston.isna().sum().sort_values()

crim       0
zn         0
indus      0
chas       0
nox        0
rm         0
age        0
dis        0
rad        0
tax        0
ptratio    0
b          0
lstat      0
medv       0
dtype: int64

In [9]:
# Anàlisi estadístic de les variables numèriques
dfBoston.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
count,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000
mean,3.620667,11.350495,11.154257,0.069307,0.554728,6.284059,68.581584,3.794459,9.566337,408.459406,18.461782,356.594376,12.668257,22.529901
std,8.608572,23.343704,6.855868,0.254227,0.115990,0.703195,28.176371,2.107757,8.707553,168.629992,2.162520,91.367787,7.139950,9.205991
min,0.009060,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082210,0.000000,5.190000,0.000000,0.449000,5.885000,45.000000,2.100000,4.000000,279.000000,17.400000,375.330000,7.010000,17.000000
50%,0.259150,0.000000,9.690000,0.000000,0.538000,6.208000,77.700000,3.199200,5.000000,330.000000,19.100000,391.430000,11.380000,21.200000
75%,3.678220,12.500000,18.100000,0.000000,0.624000,6.625000,94.100000,5.211900,24.000000,666.000000,20.200000,396.210000,16.960000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


### Divisió en train i test

In [10]:
# Divisió de les dades en conjunt train i en conjunt test (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(
                                        dfBoston.drop('medv', axis = 'columns'),
                                        dfBoston['medv'],
                                        train_size   = .8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

In [11]:
X_train

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat
215,0.04560,0.0,13.89,1,0.550,5.888,56.0,3.1121,5,276.0,16.4,392.80,13.51
414,18.08460,0.0,18.10,0,0.679,6.434,100.0,1.8347,24,666.0,20.2,27.25,29.05
411,18.81100,0.0,18.10,0,0.597,4.628,100.0,1.5539,24,666.0,20.2,28.79,34.37
467,15.57570,0.0,18.10,0,0.580,5.926,71.0,2.9084,24,666.0,20.2,368.74,18.13
337,0.03306,0.0,5.19,0,0.515,6.059,37.3,4.8122,5,224.0,20.2,396.14,8.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0.13642,0.0,10.59,0,0.489,5.891,22.3,3.9454,4,277.0,18.6,396.90,10.87
53,0.01360,75.0,4.00,0,0.410,5.888,47.6,7.3197,3,469.0,21.1,396.90,14.80
294,0.12932,0.0,13.92,0,0.437,6.678,31.1,5.9604,4,289.0,16.0,396.90,6.27
211,0.21719,0.0,10.59,1,0.489,5.807,53.8,3.6526,4,277.0,18.6,390.94,16.03


In [12]:
X_test

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat
64,0.03584,80.0,3.37,0,0.398,6.290,17.8,6.6115,4,337.0,16.1,396.90,4.67
401,9.59571,0.0,18.10,0,0.693,6.404,100.0,1.6390,24,666.0,20.2,376.11,20.31
479,5.82401,0.0,18.10,0,0.532,6.242,64.7,3.4242,24,666.0,20.2,396.90,10.74
124,0.16902,0.0,25.65,0,0.581,5.986,88.4,1.9929,2,188.0,19.1,385.02,14.81
131,0.59005,0.0,21.89,0,0.624,6.372,97.9,2.3274,4,437.0,21.2,385.76,11.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2.24236,0.0,19.58,0,0.605,5.854,91.8,2.4220,5,403.0,14.7,395.11,11.64
357,5.20177,0.0,18.10,1,0.770,6.127,83.4,2.7227,24,666.0,20.2,395.43,11.48
192,0.02187,60.0,2.93,0,0.401,6.800,9.9,6.2196,1,265.0,15.6,393.37,5.03
308,0.34940,0.0,9.90,0,0.544,5.972,76.7,3.1025,4,304.0,18.4,396.24,9.97


In [13]:
y_train

215    23.3
414     7.2
411    17.9
467    19.1
337    20.6
       ... 
204    22.6
53     18.9
294    28.6
211    22.4
303    36.1
Name: medv, Length: 404, dtype: float64

In [14]:
y_train.describe()

count    404.000000
mean      22.640842
std        9.328767
min        5.000000
25%       17.100000
50%       21.150000
75%       25.025000
max       50.000000
Name: medv, dtype: float64

In [15]:
y_test.describe()

count    101.000000
mean      22.086139
std        8.728024
min        5.000000
25%       16.600000
50%       21.400000
75%       24.100000
max       50.000000
Name: medv, dtype: float64

És important que la distribució (variança) de la variable resposta sigui semblant en el conjunt de train i en el de test, com es pot observar en aquest cas.

### Estandarització (preprocessat de dades)

In [16]:
numeric_cols = X_train.select_dtypes(include=['float64', 'int']).columns.to_list()

preprocessor = ColumnTransformer(
                   [('scale', StandardScaler(), numeric_cols)]
               )  

In [17]:
X_train_prep = preprocessor.fit_transform(X_train)
X_test_prep  = preprocessor.transform(X_test)

In [18]:
X_train_prep

array([[-0.40258847, -0.49244228,  0.43579432, ..., -0.94377342,
         0.40738847,  0.11828465],
       [ 1.65670153, -0.49244228,  1.05238535, ...,  0.81992829,
        -3.42612885,  2.28991275],
       [ 1.73962566, -0.49244228,  1.05238535, ...,  0.81992829,
        -3.4099789 ,  3.033353  ],
       ...,
       [-0.39303119, -0.49244228,  0.44018808, ..., -1.12942623,
         0.45038511, -0.89346487],
       [-0.38300016, -0.49244228, -0.04751932, ...,  0.07731704,
         0.38788268,  0.47044056],
       [-0.40149826,  0.91625645, -1.27923679, ..., -0.01550936,
         0.41661702, -0.80123356]])

In [19]:
X_train_prep.mean(axis=0)

array([-8.79384574e-18, -5.93584587e-17, -2.50074988e-16, -4.83661516e-17,
        9.66223801e-16, -2.02258452e-16, -1.75876915e-16, -2.85799987e-17,
        1.31907686e-17,  9.23353803e-17,  2.14569836e-15, -4.83661516e-16,
        9.67323031e-17])

In [20]:
X_train_prep.std(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [21]:
X_test_prep 

array([[-0.40370265,  2.92258495, -1.10495096, ..., -1.08301303,
         0.45038511, -1.11705592],
       [ 0.68762973, -0.49244228,  1.05238535, ...,  0.81992829,
         0.2323607 ,  1.06854662],
       [ 0.25706129, -0.49244228,  1.05238535, ...,  0.81992829,
         0.45038511, -0.26880736],
       ...,
       [-0.40529743,  2.06882814, -1.16939278, ..., -1.31507905,
         0.41336605, -1.06674793],
       [-0.36790737, -0.49244228, -0.1485758 , ..., -0.01550936,
         0.4434637 , -0.37641056],
       [-0.39475155, -0.49244228,  0.43579432, ..., -0.94377342,
         0.41724624, -0.30234602]])

In [22]:
X_test_prep.mean(axis=0)

array([ 0.02766378, -0.03957078,  0.1756054 ,  0.04955842,  0.04987991,
       -0.0862656 ,  0.0025375 , -0.038384  ,  0.08846652,  0.09194629,
        0.06582863,  0.13850322,  0.00327915])

In [23]:
X_test_prep.std(axis=0)

array([0.90479845, 0.97667809, 1.00321782, 1.08141877, 0.94097181,
       0.84804701, 1.09477617, 1.07477937, 1.02739485, 1.01639776,
       1.01171899, 0.75235439, 0.98378901])

En els outputs dels conjunts train i test, es pot observar que les dades escalades tenen mitjana zero i variança la unitat.

In [24]:
# Conversió del numpy array (output) en dataframe i afegir el nom de les columnes

labels = np.concatenate([numeric_cols]) 
data_train_prep = preprocessor.transform(X_train)
dfXTrainPrep = pd.DataFrame(data_train_prep, columns=labels)
dfXTrainPrep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   crim     404 non-null    float64
 1   zn       404 non-null    float64
 2   indus    404 non-null    float64
 3   chas     404 non-null    float64
 4   nox      404 non-null    float64
 5   rm       404 non-null    float64
 6   age      404 non-null    float64
 7   dis      404 non-null    float64
 8   rad      404 non-null    float64
 9   tax      404 non-null    float64
 10  ptratio  404 non-null    float64
 11  b        404 non-null    float64
 12  lstat    404 non-null    float64
dtypes: float64(13)
memory usage: 41.2 KB


In [25]:
dfXTrainPrep.head(4)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat
0,-0.402588,-0.492442,0.435794,3.736705,-0.030366,-0.565325,-0.455253,-0.336762,-0.510472,-0.771018,-0.943773,0.407388,0.118285
1,1.656702,-0.492442,1.052385,-0.267615,1.070296,0.190236,1.138620,-0.952820,1.687162,1.553233,0.819928,-3.426129,2.289913
2,1.739626,-0.492442,1.052385,-0.267615,0.370651,-2.308928,1.138620,-1.088243,1.687162,1.553233,0.819928,-3.409979,3.033353
3,1.370291,-0.492442,1.052385,-0.267615,0.225602,-0.512740,0.088113,-0.435001,1.687162,1.553233,0.819928,0.155072,0.763904


In [26]:
data_test_prep = preprocessor.transform(X_test)
dfXTestPrep = pd.DataFrame(data_test_prep, columns=labels)
dfXTestPrep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   crim     101 non-null    float64
 1   zn       101 non-null    float64
 2   indus    101 non-null    float64
 3   chas     101 non-null    float64
 4   nox      101 non-null    float64
 5   rm       101 non-null    float64
 6   age      101 non-null    float64
 7   dis      101 non-null    float64
 8   rad      101 non-null    float64
 9   tax      101 non-null    float64
 10  ptratio  101 non-null    float64
 11  b        101 non-null    float64
 12  lstat    101 non-null    float64
dtypes: float64(13)
memory usage: 10.4 KB


In [27]:
dfXTestPrep.head(4)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat
0,-0.403703,2.922585,-1.104951,-0.267615,-1.327269,-0.009032,-1.839024,1.350913,-0.626137,-0.407482,-1.083013,0.450385,-1.117056
1,0.687630,-0.492442,1.052385,-0.267615,1.189747,0.148722,1.138620,-1.047202,1.687162,1.553233,0.819928,0.232361,1.068547
2,0.257061,-0.492442,1.052385,-0.267615,-0.183946,-0.075455,-0.140101,-0.186243,1.687162,1.553233,0.819928,0.450385,-0.268807
3,-0.388499,-0.492442,2.158148,-0.267615,0.234134,-0.429711,0.718417,-0.876524,-0.857467,-1.295465,0.309383,0.325800,0.299952


### Model de regressió linial múltiple
#### Regressió linial en Statsmodels

In [28]:
X = X_train_prep
y = y_train

# Ajusto un model linial
model = sm.OLS(y, X).fit()
predictions = model.predict(X_test_prep)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   medv   R-squared (uncentered):                   0.106
Model:                            OLS   Adj. R-squared (uncentered):              0.076
Method:                 Least Squares   F-statistic:                              3.559
Date:                Sun, 08 Jan 2023   Prob (F-statistic):                    2.70e-05
Time:                        10:07:29   Log-Likelihood:                         -1842.6
No. Observations:                 404   AIC:                                      3711.
Df Residuals:                     391   BIC:                                      3763.
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -1.0471      1.540     -0.680      0.497      -4.076       1.981
x2             1.1219      1.747      0.642      0.521      -2.314       4.557
x3             0.1221      2.420      0.050      0.960      -4.636       4.881
x4             0.7161      1.218      0.588      0.557      -1.678       3.111
x5            -2.1168      2.458     -0.861      0.390      -6.949       2.715
x6             2.5233      1.616      1.562      0.119      -0.654       5.700
x7            -0.3054      2.044     -0.149      0.881      -4.323       3.713
x8            -3.4107      2.299     -1.484      0.139      -7.930       1.108
x9             3.1559      3.217      0.981      0.327      -3.169       9.480
x10           -2.5444      3.563     -0.714      0.476      -9.549       4.460
x11           -2.0605      1.570     -1.312      0.190      -5.148       1.027
x12            0.9910      1.387      0.715      0.475      -1.735       3.717
x13           -3.6475      2.003     -1.821      0.069      -7.585       0.290
==============================================================================
Omnibus:                      144.214   Durbin-Watson:                   0.097
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              595.691
Skew:                           1.532   Prob(JB):                    4.44e-130
Kurtosis:                       8.099   Cond. No.                         9.98
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
print("Les prediccions són:", predictions)

Les prediccions són: [  7.92892388  -4.23909543   1.10208123   0.29785516  -3.06932243
   4.56371537  10.08983405 -15.81587425  -1.30310499   1.70762557
  -3.76635893   3.36790516   0.96514877   7.11944112  -5.77075316
  -6.37765821  -2.23848538   2.48132018  -7.57291963   3.1138302
  13.27208159  -1.86700796  -2.54082474 -11.22187734   9.78699169
  -4.66290857  -5.01077005  11.16702651  -2.34662659   7.0739313
  -1.30540039  -3.48597494  14.74727141  -9.84885755  -2.28965244
  -3.00880482   6.48666396  -1.95491626  -0.62551713 -15.48594198
  -6.33060646 -11.17521244  -4.43650438  -3.06654842  -0.93187162
   2.16076629  -8.12606355   1.30034338  -8.90399744  12.00592086
 -16.24507754  -6.32958099  -1.29190976   9.34734951  -3.28672382
  -0.65289488   4.67874638 -15.77521227  -9.73247739  -4.34185087
   5.83730934  -4.7382808   -1.10984557   9.91850773  -1.6417848
  -0.79844199  -3.01159875  -8.04042117  14.48660372   2.79645713
 -13.92816148  -7.8273415    0.33771601  -0.36472295  -0.1

In [30]:
print("Els paràmetres són:\n", model.params)

Els paràmetres són:
 x1    -1.047075
x2     1.121890
x3     0.122137
x4     0.716147
x5    -2.116845
x6     2.523340
x7    -0.305400
x8    -3.410711
x9     3.155933
x10   -2.544407
x11   -2.060516
x12    0.990953
x13   -3.647521
dtype: float64


#### Regressió linial en SKLearn

In [31]:
# Defineixo les variables independents (dades entrenament estandaritzades) i la variable resposta 
X = X_train_prep
y = y_train

# Ajusto un model linial
lm = linear_model.LinearRegression()
model = lm.fit(X,y)

# Faig prediccions de la variable resposta (medv) amb les dades de prova estandaritzades
predictions = lm.predict(X_test_prep)
print(predictions)

[30.56976547 18.40174615 23.74292282 22.93869674 19.57151915 27.20455695
 32.73067564  6.82496733 21.33773659 24.34846716 18.87448265 26.00874674
 23.60599036 29.7602827  16.87008843 16.26318337 20.40235621 25.12216176
 15.06792196 25.75467178 35.91292317 20.77383362 20.10001685 11.41896424
 32.42783328 17.97793302 17.63007153 33.80786809 20.294215   29.71477288
 21.33544119 19.15486665 37.388113   12.79198403 20.35118914 19.63203677
 29.12750554 20.68592533 22.01532445  7.15489961 16.31023513 11.46562915
 18.2043372  19.57429317 21.70896997 24.80160788 14.51477803 23.94118497
 13.73684414 34.64676245  6.39576405 16.3112606  21.34893182 31.98819109
 19.35411777 21.9879467  27.31958796  6.86562931 12.9083642  18.29899071
 28.47815092 17.90256078 21.53099602 32.55934931 20.99905678 21.8423996
 19.62924283 14.60042042 37.1274453  25.43729871  8.7126801  14.81350008
 22.97855759 22.27611863 22.52688184 21.76965907 22.37697216 19.5763075
 31.73115604 32.32911603 36.09971901 23.98523958 36.7

In [32]:
predictions.shape

(101,)

In [33]:
print("La intersecció és:", model.intercept_)
print("Els paràmetres són:", model.coef_)

La intersecció és: 22.640841584158423
Els paràmetres són: [-1.04707463  1.1218896   0.12213716  0.71614662 -2.11684451  2.52333994
 -0.30539988 -3.41071064  3.15593339 -2.54440713 -2.06051617  0.9909535
 -3.64752065]


In [58]:
# Procediment anàleg al codi de sobre, però les dades estan en forma de dataframe
X = dfXTrainPrep
y = y_train

lm = linear_model.LinearRegression()
model = lm.fit(X,y)

predictions = lm.predict(dfXTestPrep)
print(predictions[0:5])   # Mostro les 5 primeres prediccions de la variable resposta

[30.56976547 18.40174615 23.74292282 22.93869674 19.57151915]


## Exercici 2
Compara’ls en base al MSE i al R^2.

#### MSE 

In [35]:
# Statsmodels

X = X_train_prep
y = y_train
model = sm.OLS(y, X).fit()
predictions = model.predict(X_test_prep)
print("MSE:", round(model.mse_model,3), "\nRMSE:", round(math.sqrt(model.mse_model),3))

MSE: 1970.995 
RMSE: 44.396


In [36]:
# SKLearn

X = X_train_prep
y = y_train
lm = linear_model.LinearRegression()
model = lm.fit(X,y)
predictions = lm.predict(X_test_prep)
print("MSE:", round(mean_squared_error(y_test, predictions),3),"\nRMSE:",
      round(math.sqrt(mean_squared_error(y_test, predictions)),3))

MSE: 16.556 
RMSE: 4.069


El valor més baix del MSE (o RMSE) indica errors menors i, per tant, un millor ajust del model de regressió, així que el model en SKLearn és el millor.

#### R^2 

In [37]:
# Statsmodels

X = X_train_prep
y = y_train
model = sm.OLS(y, X).fit()
#predictions = model.predict(X_test_prep)

print("R^2:", round(model.rsquared, 3), "\nR^2 ajustat:", round(model.rsquared_adj, 3))

R^2: 0.106 
R^2 ajustat: 0.076


In [38]:
# SKLearn

X = X_train_prep
y = y_train

lm = linear_model.LinearRegression()
model = lm.fit(X,y)

In [39]:
print("R^2:", round(lm.score(X,y), 3))

R^2: 0.731


El valor de R Square (R^2) en SKLearn al ser més gran que el del Statsmodels indica que el model en SKLearn presenta un millor ajust entre la predicció i el valor real.

## Exercici 3
Entrena’ls utilitzant els diferents paràmetres que admeten per intentar millorar-ne la predicció.

In [40]:
# Per millorar el models de regressió vull aplicar el "Gradient Descent" per a regressió múltiple (SKLearn),
# però al fer l'ajust de les dades m`ha donat un error que no he sabut resoldre. 

def generateXvector(X):
    vectorX = np.c_[np.ones((len(X), 1)), X]
    return vectorX

In [41]:
def theta_init(X):
    theta = np.random.randn(len(X[0])+1, 1)
    return theta

In [42]:
def Multivariable_Linear_Regression(X,y,learningrate, iterations):
    y_new = np.reshape(y, (len(y), 1))   
    cost_lst = []
    vectorX = generateXvector(X)
    theta = theta_init(X)
    m = len(X)
    for i in range(iterations):
        gradients = 2/m * vectorX.T.dot(vectorX.dot(theta) - y_new)
        theta = theta - learningrate * gradients
        y_pred = vectorX.dot(theta)
        cost_value = 1/(2*len(y))*((y_pred - y)**2) 
        #Calculate the loss for each training instance
        total = 0
        for i in range(len(y)):
            total += cost_value[i][0] 
            #Calculate the cost function for each iteration
        cost_lst.append(total)
    plt.plot(np.arange(1,iterations),cost_lst[1:], color = 'red')
    plt.title('Cost function Graph')
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost')
    return theta

In [43]:
# Regressió en SKLearn
X = X_train_prep
y = y_train
lm = linear_model.LinearRegression()
model = lm.fit(X,y)
print("La intersecció és:", model.intercept_)
print("Els paràmetres són:", model.coef_)

La intersecció és: 22.640841584158423
Els paràmetres són: [-1.04707463  1.1218896   0.12213716  0.71614662 -2.11684451  2.52333994
 -0.30539988 -3.41071064  3.15593339 -2.54440713 -2.06051617  0.9909535
 -3.64752065]


In [44]:
# Ajustament de les dades amb el Gradient Descent.
from sklearn.linear_model import LinearRegression

X = X_train_prep
y = y_train
Multivariable_Linear_Regression(X,y, 0.03, 30000)

ValueError: Data must be 1-dimensional

## Exercici 4
Compara el seu rendiment emprant l’aproximació traint/test o emprant totes les dades (validació interna).

In [45]:
# Implemento la classe GridSearchCV. Utilitzo una regressió LASSO, en què cal fixar valors d'alpha.

from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

X = X_train_prep
y = y_train

alphas = np.array([1, 0.5, 0.2, 0.1, 0.05, 0.02, 0.01])
model = Lasso()
grid = GridSearchCV(estimator = model,
                    param_grid = dict(alpha = alphas),
                    cv = 3)
print(grid.fit(X, y))
print(grid.best_params_)

GridSearchCV(cv=3, estimator=Lasso(),
             param_grid={'alpha': array([1.  , 0.5 , 0.2 , 0.1 , 0.05, 0.02, 0.01])})
{'alpha': 0.01}


Els millors paràmetres del model són per a alpha igual a 0.01.

In [46]:
# Anàlisi detallat del rendiment
grid.cv_results_

{'mean_fit_time': array([0.00433683, 0.00521517, 0.        , 0.00521851, 0.        ,
        0.        , 0.        ]),
 'std_fit_time': array([0.00613321, 0.00737536, 0.        , 0.00738008, 0.        ,
        0.        , 0.        ]),
 'mean_score_time': array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.00520953, 0.        ]),
 'std_score_time': array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.00736738, 0.        ]),
 'param_alpha': masked_array(data=[1.0, 0.5, 0.2, 0.1, 0.05, 0.02, 0.01],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'alpha': 1.0},
  {'alpha': 0.5},
  {'alpha': 0.2},
  {'alpha': 0.1},
  {'alpha': 0.05},
  {'alpha': 0.02},
  {'alpha': 0.01}],
 'split0_test_score': array([0.63884832, 0.6655122 , 0.69955475, 0.70770182, 0.70890062,
        0.70694874, 0.70600486]),
 'split1_test_score': array([0.61674304, 0.61826001, 0.64475596

In [47]:
# Anàleg al cas anterior, però en aquest codi s'avalua si és millor un model amb terme independent o sense.

fit_intercept = [True, False]

grid = GridSearchCV(estimator = model,
                    param_grid = dict(alpha = alphas,
                                      fit_intercept = fit_intercept),
                    cv = 3)

print(grid.fit(X, y))
print(grid.best_params_)

GridSearchCV(cv=3, estimator=Lasso(),
             param_grid={'alpha': array([1.  , 0.5 , 0.2 , 0.1 , 0.05, 0.02, 0.01]),
                         'fit_intercept': [True, False]})
{'alpha': 0.01, 'fit_intercept': True}


In [48]:
# Anàlisi detallat del rendiment
grid.cv_results_

{'mean_fit_time': array([0.0025413 , 0.00208036, 0.00181556, 0.00106986, 0.0003802 ,
        0.00417431, 0.00522518, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.00520929, 0.        ]),
 'std_fit_time': array([6.50219906e-04, 1.63060801e-05, 5.32006935e-04, 1.49036511e-05,
        5.37681395e-04, 5.90336858e-03, 7.38952266e-03, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        7.36704434e-03, 0.00000000e+00]),
 'mean_score_time': array([0.00034984, 0.00104197, 0.00109458, 0.00104507, 0.00035183,
        0.        , 0.        , 0.        , 0.00519896, 0.        ,
        0.00520929, 0.        , 0.        , 0.        ]),
 'std_score_time': array([4.94747806e-04, 2.65900560e-05, 3.25159590e-05, 3.01404422e-05,
        4.97557596e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        7.35243343e-03, 0.00000000e+00, 7.36704434e-03, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00]),
 'param_alpha': masked_arr

## Exercici 5
No facis servir la variable del nombre d'habitacions (RM) a l’hora de fer prediccions.

In [49]:
# Regressió linial en SKLearn (realitzat a l'exercici 1)

X = X_train_prep
y = y_train

lm = linear_model.LinearRegression()
model = lm.fit(X,y)

# Faig prediccions de la variable resposta (medv) amb les dades de prova estandaritzades
predictions = lm.predict(X_test_prep)
print(predictions)

[30.56976547 18.40174615 23.74292282 22.93869674 19.57151915 27.20455695
 32.73067564  6.82496733 21.33773659 24.34846716 18.87448265 26.00874674
 23.60599036 29.7602827  16.87008843 16.26318337 20.40235621 25.12216176
 15.06792196 25.75467178 35.91292317 20.77383362 20.10001685 11.41896424
 32.42783328 17.97793302 17.63007153 33.80786809 20.294215   29.71477288
 21.33544119 19.15486665 37.388113   12.79198403 20.35118914 19.63203677
 29.12750554 20.68592533 22.01532445  7.15489961 16.31023513 11.46562915
 18.2043372  19.57429317 21.70896997 24.80160788 14.51477803 23.94118497
 13.73684414 34.64676245  6.39576405 16.3112606  21.34893182 31.98819109
 19.35411777 21.9879467  27.31958796  6.86562931 12.9083642  18.29899071
 28.47815092 17.90256078 21.53099602 32.55934931 20.99905678 21.8423996
 19.62924283 14.60042042 37.1274453  25.43729871  8.7126801  14.81350008
 22.97855759 22.27611863 22.52688184 21.76965907 22.37697216 19.5763075
 31.73115604 32.32911603 36.09971901 23.98523958 36.7

In [62]:
# Regressió linial anàloga al cas anterior sense usar la variable rm (nombre d'habitacions)

X = dfXTrainPrep.drop(["rm"], axis=1)
y = y_train
XTest = dfXTestPrep.drop(["rm"], axis=1)

lm = linear_model.LinearRegression()
model = lm.fit(X,y)

predictions = lm.predict(XTest)
print(predictions[0:5])      # Mostro les 5 primeres prediccions de la variable resposta

[32.73002792 16.61548817 24.05820958 23.20726165 18.42851187]
